파이썬의 함수는 '일급 객체' 이다. '일급 객체'는 다음과 같은 작업을 수행할 수 있는 프로그램 객체를 의미한다.
- 런타임에 생성할 수 있다.
- 데이터 구조체의 변수나 요소에 할당할 수 있다.
- 함수 인수로 전달할 수 있다.
- 함수 결과로 반환할 수 있다.

# 5.1 함수를 객체처럼 다루기

① 함수를 생성하고 호출한 뒤 ② `__doc__` 속성을 읽고 ③ 만든 함수가 `function` 클래스의 객체인지 확인해 보자. 

In [1]:
def factorial(n):
    '''returns n!'''
    return 1 if n < 2 else n * factorial(n-1)

factorial(42)

1405006117752879898543142606244511569936384000000000

In [2]:
factorial.__doc__

'returns n!'

In [3]:
help(factorial)

Help on function factorial in module __main__:

factorial(n)
    returns n!



In [4]:
type(factorial)

function

함수 객체의 '일급' 본질을 보여주기 위해, 함수를 다른 이름으로 사용한 다음 다른 함수의 인수로 전달해 보자.

In [5]:
fact = factorial
fact

<function __main__.factorial(n)>

In [6]:
map(factorial, range(11))

In [7]:
list(map(fact, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

일급 함수가 있으면 함수형 스타일로 프로그래밍 할 수 있다. 함수형 프로그래밍의 주요 특징은 '고위 함수'이다.

# 5.2 고위 함수

① 함수를 인수로 받거나 ② 함수를 결과로 반환하는 함수

In [8]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

`sorted` 함수는 선택적인 `key` 인수로 `len` 함수를 전달받는 고위 함수이다.

대표적인 고위 함수에는 `map()`, `filter()`, `reduce()`, `apply()`가 있는데, `map()`, `filter()`, `reduce()`에는 대안이 존재한다. 리스트 컴프리헨션이나 제너레이터 표현식을 사용하면 가독성이 더 좋아진다.

## 5.2.1 `map()`, `filter()`, `reduce()`의 대안

### `map()`, `filter()`의 대안

In [9]:
list(map(fact, range(6)))

[1, 1, 2, 6, 24, 120]

In [10]:
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [11]:
list(map(factorial, filter(lambda n: n%2, range(6))))

[1, 6, 120]

In [12]:
[factorial(n) for n in range(6) if n % 2]

[1, 6, 120]

### `reduce()`의 대안

`sum()`을 사용하는 것이 가독성과 성능 면에서 낫다.

In [13]:
from functools import reduce
from operator import add
reduce(add, range(100))

4950

In [14]:
sum(range(100))

4950

연속된 항목에 어떤 연산을 적용해서, 이전 결과를 누적시키면서 일련의 값을 하나의 값으로 'reduction' 한다.

# 5.3 익명 함수

고위 함수의 인수로 사용하는 방법 외에 lambda 함수는 거의 쓰이지 않는다. while, try 등의 구문을 사용할 수 없기 때문에 가독성이 떨어지고 사용하기 까다롭다.

In [15]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

# 5.4 일곱 가지 맛의 콜러블 객체

호출할 수 있는 객체인지 알아보기 위해서는 `callable()` 내장 함수를 이용하면 된다. 콜러블의 종류는 7가지이다.

- 사용자 정의 함수
- 내장 함수
- 내장 메서드
- 메서드
- 클래스
- 클래스 객체
- 제너레이터 함수

🤔 내장 함수 vs 내장 메서드 / 클래스 vs 클래스 객체

# 5.5 사용자 정의 콜러블형

`__call__()` 인스턴스 메서드를 구현하면 모든 파이썬 객체를 함수처럼 동작하게 할 수 있다.

In [16]:
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items) # 1
        random.shuffle(self._items) # 2
    
    def pick(self): # 3
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage') # 4
    
    def __call__(self): # 5
        return self.pick()

1. `__init__()`은 반복 가능한 객체를 받는다.

In [17]:
bingo = BingoCage(10)

TypeError: 'int' object is not iterable

반복 가능한 객체가 아니라면, `TypeError`가 뜬다. `self._items = list(items)` 처럼 **지역**에 **사본**을 만들었는데, 이렇게 하면 인수로 전달된 리스트에 예기치 않은 부작용이 생기지 않도록 예방할 수 있다.

2. `self._items`가 리스트이므로, `shuffle()` 메서드의 실행을 보장할 수 있다.

In [18]:
help(random.shuffle)

Help on method shuffle in module random:

shuffle(x, random=None) method of random.Random instance
    Shuffle list x in place, and return None.
    
    Optional argument random is a 0-argument function returning a
    random float in [0.0, 1.0); if it is the default None, the
    standard random.random will be used.



In [19]:
print(random.shuffle([x for x in range(10)]))

None


In [20]:
original = [x for x in range(10)]
random.shuffle(original)
original

[3, 1, 0, 5, 7, 6, 8, 4, 2, 9]

(원본을 변경하는 `random.shuffle`)

`random.shuffle`은 input으로 list를 받아 `None`을 반환한다. 원본 리스트를 copy해 오지 않고, 리스트 자체를 변경하기 때문이다. 파이썬 API는 원본의 구조체를 바로 변경하는 경우에는 대개 `None`을 반환한다.     

파이썬이 이렇게 하는 이유는 효율성 때문이다. 매우 큰 리스트를 랜덤하게 정렬해야 할 필요가 있는 경우, 원본 리스트를 굳이 유지할 이유가 없다면 copy는 부담이 되기 때문이다.

In [21]:
original = [x for x in range(10)]
copy = list(original)

random.shuffle(copy)
print(original)
print(copy)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[8, 0, 9, 1, 3, 7, 4, 5, 6, 2]


(원본을 따로 남기고 싶다면 copy 한 뒤 사용하자)

In [22]:
bingo = BingoCage(range(10))

In [23]:
bingo._items

[2, 9, 4, 5, 0, 3, 8, 6, 1, 7]

3. 핵심 메서드

In [24]:
while True:
    print(original.pop())

9
8
7
6
5
4
3
2
1
0


IndexError: pop from empty list

4. `self._items`가 비어 있으면 사용자 정의 메시지를 담은 예외를 발생시킨다.

In [25]:
bingo = BingoCage(range(10))

In [26]:
while True:
    print(bingo._items.pop())

0
5
6
9
4
3
2
8
1
7


IndexError: pop from empty list

In [27]:
bingo = BingoCage(range(10))

while True:
    print(bingo.pick())

4
0
6
1
7
9
5
2
8
3


LookupError: pick from empty BingoCage

🤔 원래 `IndexError`가 발생하는데, 따로 `LookupError`를 정의하여 발생시키는 이유가 있을까?

5. `bingo.pick()`에 대한 단축 형태로 `bingo()`를 정의한다.      

In [28]:
bingo = BingoCage(range(10))

while True:
    print(bingo())

7
3
1
9
6
4
0
8
2
5


LookupError: pick from empty BingoCage

`bingo` 객체를 함수처럼 호출할 수 있는 이유는 `__call__()` 메서드를 구현했기 때문이다.

In [29]:
callable(bingo)

True

`BingoCage` 의 경우, 객체를 함수처럼 호출할 때마다(e.g. `bingo()`) 항목을 하나 꺼낸 후 변경된 상태를 유지해야 한다. 이런 경우, `__call__()` 메서드를 구현하는 것이 유리하다고 한다.    

🤔 `__call__()`의 기능은 '객체를 함수처럼 호출하기'인데, '항목을 하나 꺼낸 후 변경된 상태 유지하기'와 무슨 관련이 있는 것인가?

https://pythontutor.com/visualize.html#mode=edit

# 5.6 함수 인트로스펙션

introspection : 특정 클래스가 어떤 클래스로부터 파생되었는지, 혹은 어떤 함수가 구현되어 있는지, 객체에는 어떤 속성이 있는지에 대한 상세한 정보를 런타임에 얻거나 조작하는 기술, 파이썬에서 간단하게 `?`로 사용할 수 있다.

In [43]:
?factorial

함수 객체는 `__doc__` 이외에도 많은 속성을 가지고 있다.

In [40]:
dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [35]:
bingo = BingoCage(range(10))
bingo.__dict__

{'_items': [5, 8, 7, 2, 6, 1, 9, 4, 3, 0]}

`__dict__` 속성을 이용해서 객체에 할당된 속성을 보관한다.

In [36]:
class C: pass
obj = C()

def func(): pass
sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

- `C` : 사용자 정의 클래스
- `obj` : 사용자 정의 클래스의 인스턴스
- `func` : 사용자 정의 함수

사용자 정의 함수에는 존재하지만, 기본 클래스의 객체에는 존재하지 않는 속성들

# 5.7 위치 매개변수에서 키워드 전용 매개변수까지

In [64]:
def tag(name, *content, cls=None, **attrs):
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value) for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [65]:
tag('br')

'<br />'

위치 인수 하나만 사용해서 호출하면 빈 태그를 생성한다.

In [66]:
tag('p', 'hello')

'<p>hello</p>'

In [67]:
print(tag('p', 'hello', 'world'))

<p>hello</p>
<p>world</p>


첫 번째 이후의 인수들은 모두 `*content` 매개변수에 튜플로 전달된다.

In [68]:
tag('p', 'hello', id=33)

'<p id="33">hello</p>'

In [69]:
print(tag('p', 'hello', 'world', cls='sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


`tag` 시그니처에 명시적으로 이름이 지정되지 않은 키워드 인수(e.g., `id`, `sidebar`)들은 딕셔너리로 `**attrs`인수에 전달된다.

In [70]:
tag(content='testing', name='img')

'<img content="testing" />'

In [71]:
my_tag = {'name':'img',
         'title':'Sunset Boulevard',
         'src':'sunset.jpg',
         'cls':'framed'}
tag(**my_tag)

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

딕셔너리 안의 모든 항목(위치 매개변수 제외)을 별도의 인수로(`*content`) 전달, 명명된 매개변수 및 나머지는 `**attrs`에 전달됨.

# 5.9 함수 애너테이션

# 5.10 함수형 프로그래밍을 위한 패키지